# DS-GA 1003 Machine Learning Spring 2021
## Lab 1: 3-Feb-2020, Wednesday
## Gradients and Directional Derivatives

In this lab, you will learn to:

- use Python to compute the gradient for two simple functions
- how to empirically check the correctness of your gradient computation

---
### 1. Computing Gradients

Most numerical optimization methods require that we compute gradients of the loss function that we are attempting to minimize.  

In this section, we illustrate how to compute gradients efficiently in python for a few simple examples.  

In [1]:
# Import required packages
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing

#### 1.1 Example 1: simple vector function

Let's consider a simple function:

$$f(\mathbf{\theta}) = \theta_0^2 + 2\theta_0 \theta_1^3$$ 

The gradient of this function can be computed as following:

$$\frac{\partial}{\partial \theta_0} = 2\theta_0 + 2\theta_1^3$$

$$\frac{\partial}{\partial \theta_1} = 6\theta_0\theta_1^2$$

$$\nabla f(\mathbf{\theta}) = \begin{bmatrix} 2\theta_0 + 2\theta_1^3 \\ 6\theta_0\theta_1^2 \end{bmatrix}$$



In [2]:
def simple_vector_func(theta):
    """
    Function that returns the value and gradient for a given theta vector
    @param theta: 2d numpy array [theta_0, theta_1]
    @out: f(\theta), gradient of f(\theta)
    """
    # unpack vector
    theta_0, theta_1 = theta
    
    # compute the value
    y_val = np.power(theta_0,2) + 2*theta_0* np.power(theta_1,3)

    # compute the gradient
    gradient_theta0 = 2*theta_0 + 2*np.power(theta_1,3)
    gradient_theta1 = 6*theta_0*np.power(theta_1,2)
    gradient = np.array([gradient_theta0, gradient_theta1])
    return y_val, gradient

Let's run this function for some random $\theta$.

In [3]:
theta = np.array([2,3])
y_val, grad = simple_vector_func(theta)
print("y_val={0}, grad={1}".format(y_val, grad))

y_val=112, grad=[ 58 108]


#### 1.2 Example 2: least squares

Say that we have a linear model 

$$f(\mathbf{x}) = \theta \cdot \mathbf{x} = \theta_1 x_1 + \theta_2 x_2 + ... + \theta_d x_d$$

for a data point $\mathbf{x}_i \in \mathbb{R}^d$ and a parameter set $\theta \in \mathbb{R}^d$.

Now, given a single label $y_i \in \mathbb{R}$, we want to calcuate the $l_2$ loss:

$$J(\theta) = |f(\mathbf{x}) - y|_2^2$$

So what is the gradient of $J(\theta)$?

$$\nabla J(\theta) = 2\mathbf{x}(\mathbf{x} \cdot \mathbf{\theta} - y)$$

In [4]:
def l2_loss_linear_func(theta, x, y):
    """
    Function that returns the value and gradient for a given theta vector
    @param theta: nd numpy array [theta_1, theta_2, ..., theta_n]
    @param x: same dimension as theta
    #param y: a real number
    @out: f(\theta), gradient of f(\theta)
    """
    # compute the value
    y_val = np.power(np.dot(theta, x) - y, 2)

    # compute the gradient
    gradient = 2 * x * (np.dot(theta,x) - y)
    return y_val, gradient

In [6]:
theta = np.array([1,2,3,4,5])
x_i = np.array([1,2,3,4,5])
y_i = 99
y_val, grad = l2_loss_linear_func(theta, x_i, y_i)
print("y_val={0}, grad={1}".format(y_val, grad))

y_val=1936, grad=[ -88 -176 -264 -352 -440]


### 1.3 Testing the gradient

So far we have worked with relatively simple functions where it is straight-forward to compute the gradient. For more complex functions, the gradient computation can be notoriously difficult to debug and get right.

It is very important to test if the gradient is correct. Recall the mathematical definition of the derivative as:
$$ \frac{\partial}{\partial \theta} f(\theta) = \text{lim}_{\epsilon \to 0} \frac{f(\theta + \epsilon) - f(\theta - \epsilon) }{2 \epsilon}$$

We can approximate the gradient (left hand side) using the equation on the right hand side by setting $\epsilon$ to a small constant, say around $10^{−4}$.

Now let's expand this method to deal with vector input $\theta \in \mathbb{R}^d$. Let's say we want to verify out gradient at demension $i$ $(\nabla f(\theta))_i$. We can make use of one-hot vector $e_i$ in which all dimension except the $i$th are 0 and the $i$th dimension has a value of 1: 

$$e_i = \begin{bmatrix} 0\\ 0\\ ...\\ 1\\ ... \\0 \end{bmatrix}$$

The gradient at $i$th dimension can be then approximated as 

$$ [\nabla f(\theta)]^{(i)} \approx \frac{f(\theta + \epsilon e_i)-f(\theta - \epsilon e_i)}{2 \epsilon} $$

We can then calculate the approximation for all dimension.


More information about the gradient checker can be found at http://deeplearning.stanford.edu/tutorial/supervised/DebuggingGradientChecking/.

In [8]:
def approxiate_gradient(f_theta, theta, epsilon=1e-5):
    """
    Function that performs the gradient approximation using the above logic
    @param: f_theta, a function that calculates f(theta) and gradient
    @param: theta, a numpy array
    @output: the approximated value of the gradient at theta
    """
    n_dim = len(theta)
    gradient = np.zeros(n_dim)
    
    # iterate through all n dimension
    for i in range(n_dim):
        # construct e_i
        e_i = np.zeros(n_dim)
        e_i[i] = 1
        # calcualte f(theta+epsilon) and f(theta-epsilon)
        f_plus, _ = f_theta(theta + e_i * epsilon)
        f_minus, _ = f_theta(theta - e_i * epsilon)
        # calculate the approximated gradient at dimension i
        gradient[i] = (f_plus-f_minus)/(2*epsilon)
    return gradient

Let's use this function to verify that our gradient calculation in the two examples above is correct.

In [9]:
theta = np.array([2,3])
_, grad = simple_vector_func(theta)
grad_approx = approxiate_gradient(simple_vector_func, theta)
print("Gradient (calculated by hand): ", grad)
print("Gradient (approximated): ", grad_approx)

Gradient (calculated by hand):  [ 58 108]
Gradient (approximated):  [ 58. 108.]


In [10]:
theta = np.array([1,2,3,4,5])
x_i = np.array([1,2,3,4,5])
y_i = 99
y_val, grad = l2_loss_linear_func(theta, x_i, y_i)
# here because l2_loss_linear_func also takes in x_i and y_i
# we need to use lambda to wrap the value of x_i and y_i
grad_approx = approxiate_gradient(lambda theta: l2_loss_linear_func(theta, x_i, y_i), theta)
print("Gradient (calculated by hand): ", grad)
print("Gradient (approximated): ", grad_approx)

Gradient (calculated by hand):  [ -88 -176 -264 -352 -440]
Gradient (approximated):  [ -88.00000003 -175.99999999 -264.00000002 -351.99999999 -440.00000001]


---
## References
- DS-GA 1003 Machine Learning Spring 2020